In [1]:
suppressMessages(require(Rtsne))
suppressMessages(require(cellrangerRkit))
suppressMessages(require(ggplot2))
suppressMessages(require(data.table))
suppressMessages(require(gplots))
suppressMessages(require(matrixStats))
suppressMessages(require(RColorBrewer))
require(RUVnormalize)

Loading required package: RUVnormalize
Loading required package: scran
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘scran’”

In [12]:
load("testing_RUVscale_mt_unknownRMed.Rdata") #testing_TPMscale.Rdata testing_RUVscale_mthk.Rdata
load("Seurat_out_VDJ.Rdata")
sample_gp=Exp_Seurat_CCA@meta.data$batch
cell_assignment=Exp_Seurat_CCA@meta.data$cell_assignment
rtsne_allcell=list()
rtsne_allcell$Y=Exp_Seurat_CCA@dr$tsne@cell.embeddings

In [13]:
patient_type=unique(gsub("_.*","",sample_list))
patient_assignment=rep(0,length(sample_gp))
for(i in 1:length(patient_type)){
    patient_assignment[sample_gp %in% grep(patient_type[i],sample_list)]=i
}

tissue_type=c("pBMC","I","R")
tissue_assignment=rep(0,length(sample_gp))
for(i in 1:length(tissue_type)){
    tissue_assignment[sample_gp %in% grep(tissue_type[i],sample_list)]=i
}

In [14]:
#require(repr)
#options(repr.plot.width=8, repr.plot.height=16)
pdf("TSNE_PCA_Seurat.pdf",width=10,height=25)
par(mfrow=c(5,2))

#color by library
tagcol=colorRampPalette(RColorBrewer::brewer.pal(9,"Set1"))
tagcol=tagcol(length(sample_list))
plot(rtsne_allcell$Y,pch=19,col=tagcol[sample_gp],cex=0.5,main="TSNE PBMC/Rectum label",xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+20))
legend("topright",sample_list,bty = "n",lty=0,pch=19,col=tagcol)
plot(pca_allcell$x[,1:2],pch=19,col=tagcol[sample_gp],cex=0.5,main="PCA PBMC/Rectum label",xlim=c(min(pca_allcell$x[,1]),max(pca_allcell$x[,1])+20))
legend("topright",sample_list,bty = "n",lty=0,pch=19,col=tagcol)

#color by cell type
tagcol=RColorBrewer::brewer.pal(length(cell_assignment_gps),"Set1")
coltouse=tagcol[match(cell_assignment,cell_assignment_gps)]
plot(rtsne_allcell$Y[cell_assignment==cell_assignment_gps[5],],pch=19,col=tagcol[5],cex=0.5,main="TSNE B/T cell label",xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+20))
points(rtsne_allcell$Y[cell_assignment==cell_assignment_gps[1],],pch=19,col=tagcol[1],cex=0.5)
points(rtsne_allcell$Y[cell_assignment==cell_assignment_gps[2],],pch=19,col=tagcol[2],cex=0.5)
points(rtsne_allcell$Y[cell_assignment==cell_assignment_gps[3],],pch=19,col=tagcol[3],cex=0.5)
points(rtsne_allcell$Y[cell_assignment==cell_assignment_gps[4],],pch=19,col=tagcol[4],cex=0.5)

legend("topright",cell_assignment_gps,bty = "n",lty=0,pch=19,col=tagcol)
plot(pca_allcell$x[,1:2],pch=19,col=coltouse,cex=0.5,main="PCA B/T cell label",xlim=c(min(pca_allcell$x[,1]),max(pca_allcell$x[,1])+20))
legend("topright",cell_assignment_gps,bty = "n",lty=0,pch=19,col=tagcol)


#color by tissue
tagcol=RColorBrewer::brewer.pal(length(tissue_type),"Set1")
plot(rtsne_allcell$Y,pch=19,col=tagcol[tissue_assignment],cex=0.5,main="TSNE PBMC/Rectum/Ileum\nTissue label",xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+20))
legend("topright",tissue_type,bty = "n",lty=0,pch=19,col=tagcol)
plot(pca_allcell$x[,1:2],pch=19,col=tagcol[tissue_assignment],cex=0.5,main="TSNE PBMC/Rectum/Ileum\nTissue label",xlim=c(min(pca_allcell$x[,1]),max(pca_allcell$x[,1])+20))
legend("topright",tissue_type,bty = "n",lty=0,pch=19,col=tagcol)

#color by patient
tagcol=RColorBrewer::brewer.pal(length(patient_type),"Set1")

plot(rtsne_allcell$Y,pch=19,col=tagcol[patient_assignment],cex=0.5,main="TSNE patient label",xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+20))
legend("topright",patient_type,bty = "n",lty=0,pch=19,col=tagcol)
plot(pca_allcell$x[,1:2],pch=19,col=tagcol[patient_assignment],cex=0.5,main="PCA patient label",xlim=c(min(pca_allcell$x[,1]),max(pca_allcell$x[,1])+20))
legend("topright",patient_type,bty = "n",lty=0,pch=19,col=tagcol)

#color by disease or not
tagcol=RColorBrewer::brewer.pal(2,"Set1")
diseaseOrNot=c("healthy","disease")
disease_assignment=rep(1,length(patient_assignment))
disease_assignment[patient_assignment %in% grep("U",patient_type)]=2

plot(rtsne_allcell$Y[disease_assignment==1,],pch=19,col=tagcol[1],cex=0.5,main="TSNE patient label",xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+20))
points(rtsne_allcell$Y[disease_assignment==2,],pch=19,col=tagcol[2],cex=0.5)
legend("topright",diseaseOrNot,bty = "n",lty=0,pch=19,col=tagcol[1:2])
plot(pca_allcell$x[,1:2],pch=19,col=tagcol[disease_assignment],cex=0.5,main="PCA patient label",xlim=c(min(pca_allcell$x[,1]),max(pca_allcell$x[,1])+20))
legend("topright",diseaseOrNot,bty = "n",lty=0,pch=19,col=tagcol[1:2])

dev.off()


Warning message in RColorBrewer::brewer.pal(2, "Set1"):
“minimal value for n is 3, returning requested palette with 3 different levels
”

png 
  2

In [9]:
ls()

[1] "BCR_annoation"                         
 [2] "BCR_annoation_w"                       
 [3] "cell_assignment"                       
 [4] "cell_assignment_gps"                   
 [5] "center_by_batch"                       
 [6] "coltouse"                              
 [7] "cross_combining"                       
 [8] "disease_assignment"                    
 [9] "diseaseOrNot"                          
[10] "ercc_matrix"                           
[11] "ExpressionBinaryMat"                   
[12] "ExpressionFiltered"                    
[13] "ExpressionMat"                         
[14] "ExpressionNormed"                      
[15] "Exp_Seurat"                            
[16] "Exp_Seurat_CCA"                        
[17] "genes.use"                             
[18] "genome"                                
[19] "housekeeping_gene"                     
[20] "i"                                     
[21] "mart"                                  
[22] "num_of_unknown_cell_at_most_per_lirary"
[23] "patient_assignment"                    
[24] "patient_type"                          
[25] "pca_allcell"                           
[26] "ptm"                                   
[27] "rtsne_allcell"                         
[28] "sample_gp"                             
[29] "sample_list"                           
[30] "sdY"                                   
[31] "second_to_humanReadableTime"           
[32] "ssd"                                   
[33] "t"                                     
[34] "t2g"                                   
[35] "tag_B"                                 
[36] "tagcol"                                
[37] "taggene"                               
[38] "taggene_mt"                            
[39] "taggene_posi"                          
[40] "tag_T"                                 
[41] "TCR_annoation"                         
[42] "TCR_annoation_w"                       
[43] "tissue_assignment"                     
[44] "tissue_type"                           
[45] "valid_BCR_list"                        
[46] "valid_B_list"                          
[47] "valid_cell_barcodes"                   
[48] "valid_sc_barcode"                      
[49] "valid_TCR_list"                        
[50] "valid_T_list"                          
[51] "venn_matrix"